In [1]:
from bs4 import BeautifulSoup
import requests
import nltk

In [2]:
URL = 'http://radar.oreilly.com/2010/07/louvre-industrial-age-henry-ford.html'

In [3]:
r = requests.get(URL)

In [4]:
page = BeautifulSoup(r.text)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [5]:
txt = ' '.join([p.text for p in page.find('div', attrs={'class': 'entry-content'}).find_all('p')])
txt = txt.replace('\n', '')

In [6]:
txt[:100]

'This morning I had the chance to get a tour of The Henry Ford Museum in Dearborn, MI, along with Dal'

In [7]:
# EOS Detection
sentences = nltk.tokenize.sent_tokenize(txt)
sentences

['This morning I had the chance to get a tour of The Henry Ford Museum in Dearborn, MI, along with Dale Dougherty, creator of Make: and Makerfaire, and Marc Greuther, the chief curator of the museum.',
 'I had expected a museum dedicated to the auto industry, but it’s so much more than that.',
 'As I wrote in my first stunned tweet, “it’s the Louvre of the Industrial Age.”  When we first entered, Marc took us to what he said may be his favorite artifact in the museum, a block of concrete that contains Luther Burbank’s shovel, and Thomas Edison’s signature and footprints.',
 'Luther Burbank was, of course, the great agricultural inventor who created such treasures as the nectarine and the Santa Rosa plum.',
 'Ford was a farm boy who became an industrialist; Thomas Edison was his friend and mentor.',
 'The museum, opened in 1929, was Ford’s personal homage to the transformation of the world that he was so much a part of.',
 'This museum chronicles that transformation.',
 'The machines ar

In [8]:
# Tokenization
tokens = [nltk.tokenize.word_tokenize(s) for s in sentences]
tokens[0]

['This',
 'morning',
 'I',
 'had',
 'the',
 'chance',
 'to',
 'get',
 'a',
 'tour',
 'of',
 'The',
 'Henry',
 'Ford',
 'Museum',
 'in',
 'Dearborn',
 ',',
 'MI',
 ',',
 'along',
 'with',
 'Dale',
 'Dougherty',
 ',',
 'creator',
 'of',
 'Make',
 ':',
 'and',
 'Makerfaire',
 ',',
 'and',
 'Marc',
 'Greuther',
 ',',
 'the',
 'chief',
 'curator',
 'of',
 'the',
 'museum',
 '.']

In [9]:
# POS Tagging
pos_tagged_tokens = [nltk.pos_tag(t) for t in tokens]
pos_tagged_tokens[0]

[('This', 'DT'),
 ('morning', 'NN'),
 ('I', 'PRP'),
 ('had', 'VBD'),
 ('the', 'DT'),
 ('chance', 'NN'),
 ('to', 'TO'),
 ('get', 'VB'),
 ('a', 'DT'),
 ('tour', 'NN'),
 ('of', 'IN'),
 ('The', 'DT'),
 ('Henry', 'NNP'),
 ('Ford', 'NNP'),
 ('Museum', 'NNP'),
 ('in', 'IN'),
 ('Dearborn', 'NNP'),
 (',', ','),
 ('MI', 'NNP'),
 (',', ','),
 ('along', 'IN'),
 ('with', 'IN'),
 ('Dale', 'NNP'),
 ('Dougherty', 'NNP'),
 (',', ','),
 ('creator', 'NN'),
 ('of', 'IN'),
 ('Make', 'NNP'),
 (':', ':'),
 ('and', 'CC'),
 ('Makerfaire', 'NNP'),
 (',', ','),
 ('and', 'CC'),
 ('Marc', 'NNP'),
 ('Greuther', 'NNP'),
 (',', ','),
 ('the', 'DT'),
 ('chief', 'JJ'),
 ('curator', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('museum', 'NN'),
 ('.', '.')]

In [10]:
# Chunking
pass

In [11]:
# Extraction
ne_chunks = [nltk.ne_chunk(ptt) for ptt in pos_tagged_tokens]

In [12]:
print(ne_chunks[0].pprint())

(S
  This/DT
  morning/NN
  I/PRP
  had/VBD
  the/DT
  chance/NN
  to/TO
  get/VB
  a/DT
  tour/NN
  of/IN
  The/DT
  (PERSON Henry/NNP Ford/NNP Museum/NNP)
  in/IN
  (GPE Dearborn/NNP)
  ,/,
  (ORGANIZATION MI/NNP)
  ,/,
  along/IN
  with/IN
  (PERSON Dale/NNP Dougherty/NNP)
  ,/,
  creator/NN
  of/IN
  (GPE Make/NNP)
  :/:
  and/CC
  (ORGANIZATION Makerfaire/NNP)
  ,/,
  and/CC
  (PERSON Marc/NNP Greuther/NNP)
  ,/,
  the/DT
  chief/JJ
  curator/NN
  of/IN
  the/DT
  museum/NN
  ./.)
None


In [13]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/chrisgiler_developer/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/chrisgiler_developer/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [14]:
import json
import numpy as np

In [22]:
N = 100 # Number of words to consider
CLUSTER_THRESHOLD = 5 # Distance between words to consider
TOP_SENTENCES = 2 # Number of sentences to return for a "top n" summary

def _score_sentences(sentences, important_words):
    scores = []
    sentence_idx = -1
    
    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:
        sentence_idx += 1
        word_idx = []
        
        for w in important_words:
            try:
                word_idx.append(s.index(w))
            except ValueError or e:
                pass
        word_idx.sort()

        if len(word_idx) == 0:
            continue

        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[1] - word_idx[i - 1] < CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)

        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster * significant_words_in_cluster / total_words_in_cluster
            if score > max_cluster_score:
                max_cluster_score = score
        scores.append((sentence_idx, score))
    return scores

def summarize(txt):
    sentences = [s for s in nltk.tokenize.sent_tokenize(txt)]
    normalized_sentences = [s.lower() for s in sentences]
    
    words = [w.lower() for sentence in normalized_sentences for w in nltk.tokenize.word_tokenize(sentence)]
    
    fdist = nltk.FreqDist(words)
    
    top_n_words = [w[0] for w in fdist.items() if w[0] not in nltk.corpus.stopwords.words('english')][:N]
    
    scored_sentences = _score_sentences(normalized_sentences, top_n_words)
    
    avg = np.mean([s[1] for s in scored_sentences])
    std = np.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences if score > avg + 0.5 * std]
    
    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-TOP_SENTENCES:]
    top_n_scored = sorted(top_n_scored, key=lambda s: s[0])
    
    return dict(top_n_summary=[sentences[idx] for (idx, score) in top_n_scored],
                mean_scored_summary=[sentences[idx] for (idx, score) in mean_scored])



In [47]:
txt = """Alex Irpan, a software engineer at Google, wrote an excellent article on the current difficulties of getting deep reinforcement learning to work. For example, even after weeks of optimizing hyperparameters and explotation-exploration rates, these models are still highly sensitive to initial conditions. A 30% failure rate is seen as “working.”

Irpan makes the argument that most attempts with deep RL fail but no one talks about it publicly, we only see the few cases where the problems are simplified enough to be feasible. He’s optimistic though. This is still a new field - the breakthrough Atari DQN paper was published only 3 years ago - so there is plenty of room for more research and advancement.

Deep Reinforcement Learning for Trading
From Denny Britz, a nice write up on using deep reinforcement learning for financial trading. This isn’t a tutorial with code, but rather about the difficulties and where reinforcement learning can fit in to this problem. Even if you have very limited knowledge of trading (like me), this article is a great starting point in a really interesting area.

Sky-High Wages for AI Experts
This week, Bloomberg reported on the extreme demand for AI experts. According to a report by Element AI, only 22,000 people are capable of building AI systems worldwide. Tencent recently reported the number to be around 200,000, so let’s say it’s more like 100,000 worldwide. This limited supply of AI practitioners and the high demand for them have led to salaries reaching $300,000. This demand will likely continue for the next few years as more applications for deep learning are discovered and more organizations learn what AI can do for them.

Simplifying High-Performance Computing
Much of the success of deep learning has come from the gains of high-performance computing on GPUs. So far this has mostly been done with CUDA and CuDNN, a library for deep learning build on CUDA. TensorFlow, PyTorch, and other current frameworks all run on CuDNN for efficient training and inference. As an alternative, Facebook announced Tensor Comprehensions, a C++ library that simplifies compiling machine learning applications for high-performance on GPUs and CPUs. While this still takes some computer engineering expertise, it should lower the bar for anyone looking to optimize their code.

""".replace('\n', ' ')

In [48]:
txt

'Alex Irpan, a software engineer at Google, wrote an excellent article on the current difficulties of getting deep reinforcement learning to work. For example, even after weeks of optimizing hyperparameters and explotation-exploration rates, these models are still highly sensitive to initial conditions. A 30% failure rate is seen as “working.”  Irpan makes the argument that most attempts with deep RL fail but no one talks about it publicly, we only see the few cases where the problems are simplified enough to be feasible. He’s optimistic though. This is still a new field - the breakthrough Atari DQN paper was published only 3 years ago - so there is plenty of room for more research and advancement.  Deep Reinforcement Learning for Trading From Denny Britz, a nice write up on using deep reinforcement learning for financial trading. This isn’t a tutorial with code, but rather about the difficulties and where reinforcement learning can fit in to this problem. Even if you have very limited

In [50]:
TOP_SENTENCES = 2
summarize(txt)

{'mean_scored_summary': ['Alex Irpan, a software engineer at Google, wrote an excellent article on the current difficulties of getting deep reinforcement learning to work.',
  'For example, even after weeks of optimizing hyperparameters and explotation-exploration rates, these models are still highly sensitive to initial conditions.',
  'A 30% failure rate is seen as “working.”  Irpan makes the argument that most attempts with deep RL fail but no one talks about it publicly, we only see the few cases where the problems are simplified enough to be feasible.',
  'This is still a new field - the breakthrough Atari DQN paper was published only 3 years ago - so there is plenty of room for more research and advancement.',
  'Even if you have very limited knowledge of trading (like me), this article is a great starting point in a really interesting area.'],
 'top_n_summary': ['Alex Irpan, a software engineer at Google, wrote an excellent article on the current difficulties of getting deep rei